In [ ]:
import pandas as pd
import itertools

In [ ]:
df = pd.read_csv("C:/D drive/PSG/sem8/20XW87-DATA-MINING-LAB/ca1-practice/KaggleCovidDataset-Copy1.csv").applymap(lambda x:1 if x=="Yes" else 0)

In [ ]:
df

In [ ]:
def apriori_algorithm(df, min_support=0.1):
    
    df_copy = df.copy(deep=True)
    
    f_one_itemsets = {}
    
    for col in df.columns:
        item_count = df[col].sum()/df.shape[0]
        if item_count>min_support:
            f_one_itemsets[frozenset((col,))] = item_count
            
    def generate_k_frequent_itemsets(frequent_itemset, k):
        frequent_k_itemsets = set()
        
        for i1, i2 in itertools.combinations(frequent_itemset, 2):
            union_set = i1.union(i2)
            
            if len(union_set)==k:
                subsets = [frozenset(c) for c in itertools.combinations(union_set, k-1)]
                
                if all([subset in frequent_itemset for subset in subsets]):
                    frequent_k_itemsets.add(union_set)
                
        frequent_k_itemsets_support = {}
        
        for k_itemset in frequent_k_itemsets:
            support = df_copy[list(k_itemset)].all(axis=1).mean()
            
            if support>=min_support:
                frequent_k_itemsets_support[k_itemset] = support
                
        return frequent_k_itemsets_support

    all_frequent_itemsets = {**f_one_itemsets}
    frequent_k_itemsets = f_one_itemsets
    k = 2
    print(frequent_k_itemsets)
    while len(frequent_k_itemsets)>0:
        frequent_k_itemsets = generate_k_frequent_itemsets(set(frequent_k_itemsets.keys()), k)
        print(frequent_k_itemsets)
        all_frequent_itemsets.update(frequent_k_itemsets)
        k += 1
    
    return pd.DataFrame([(support, itemset) for itemset, support in all_frequent_itemsets.items()], columns=['support', 'itemset'])

In [ ]:
def association_rules(df, min_cf=0.5):
    
    frequent_itemsets_map = {frozenset(itemset): support for support, itemset in zip(df['support'], df['itemset'])}
    associations = []
    for itemset in frequent_itemsets_map:
        
        if len(itemset)>1:
            for i in range(1, len(itemset)):
                antecendants_itemset = list(itertools.combinations(itemset, i))
                
                for antecedant in antecendants_itemset:
                    antecedant = frozenset(antecedant)
                    consequent = antecedant.difference(itemset)
                    
                    conf = frequent_itemsets_map[itemset]/frequent_itemsets_map[antecedant]
                    
                    if conf>=min_cf:
                        associations.append((antecedant, consequent, conf))
                        
                        
    return pd.DataFrame(associations, columns=['antecedants', 'consequents', 'confidence'])


In [ ]:
fdf = apriori_algorithm(df)

In [ ]:
association_rules(fdf, 0.6)

In [ ]:
apriori_algorithm(df)